#### Optimal Opponent Experiments
Author: Yemi Kelani

##### Google Drive Setup (Skip if running locally)

> To run this notebook, follow these steps:
> 1. Download the latest version of the [repository](https://github.com/yemi-kelani/artificial-intelligence/tree/master).
> 2. Upload the repsitory files to your Google Drive account under the path `Projects/artificial-intelligence`.
> 3. Open this file (`train.ipynb`) from your Google Drive and run the experiments.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
ROOT_FOLDER = "drive/MyDrive/Projects/artificial-intelligence/models/ReinforcementLearning/"
PROJECT_PATH = f"{ROOT_FOLDER}/DeepQ_TicTacToe_v2"
NOTEBOOK_LOCATION = f"{PROJECT_PATH}/experiments"

In [3]:
!cp {PROJECT_PATH}/DeepQAgent.py .
!cp {PROJECT_PATH}/TicTacToeGame.py .
!cp {ROOT_FOLDER}/Utils.py .

from DeepQAgent import DeepQAgent
from TicTacToeGame import TicTacToeGame, OPPONENT_LEVEL
from Utils import (
    train_agent,
    test_agent
)
MODEL_PATH = "drive/MyDrive/Projects/artificial-intelligence/trained_models/ReinforcementLearning/TicTacToeV2"

##### Local Setup (Skip if running remotely)
> 1. Run the following cells

In [ ]:
from models.ReinforcementLearning.DeepQ_TicTacToe_v2.DeepQAgent import DeepQAgent
from models.ReinforcementLearning.DeepQ_TicTacToe_v2.TicTacToeGame import TicTacToeGame, OPPONENT_LEVEL
from models.ReinforcementLearning.Utils import (
    train_agent,
    test_agent
)
MODEL_PATH = "../../../trained_models/ReinforcementLearning/TicTacToeV2"

##### Experiments

In [7]:
import os
import torch

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(DEVICE)

# DeepQ parameters
BATCH_SIZE     = 256
NUM_EPISODES   = 10000 if torch.cuda.is_available() else 100
STATE_SPACE    = 9
ACTION_SPACE   = 9
HIDDEN_SIZE    = 128
EPSILON        = 1.0
GAMMA          = 0.99
LEARNING_RATE  = 0.001
DROPOUT        = 0.25
TRAIN_START = 1000 # MUST BE EQUAL TO OR LESS THAN 2000 (Maxlen of replay memory)
NEGATIVE_SLOPE = 0.01

# save path
MODEL_NAME = ""

# models
BASELINE = "TicTacToe-v2-BASELINE"
NAIVE = "TicTacToe-v2-NAIVE"
AGENT = "TicTacToe-v2-AGENT"
OPTIMAL = "TicTacToe-v2-OPTIMAL"
SELF = "TicTacToe-v2-SELF"


def get_full_model_path(agent_name: str = None):
  if agent_name is None:
    return os.path.join(MODEL_PATH, MODEL_NAME + ".pt")
  return os.path.join(MODEL_PATH, agent_name + ".pt")

def supply_model(load_if_exists: bool = True, agent_name: str = None):

  agent = DeepQAgent(
      device         = DEVICE,
      epsilon        = EPSILON,
      gamma          = GAMMA,
      state_space    = STATE_SPACE,
      action_space   = ACTION_SPACE,
      hidden_size    = HIDDEN_SIZE,
      dropout        = DROPOUT,
      train_start    = TRAIN_START,
      batch_size     = BATCH_SIZE,
      negative_slope = NEGATIVE_SLOPE
  )

  full_model_path = get_full_model_path(agent_name)
  if load_if_exists and os.path.exists(full_model_path):
    print("Loading Model Parameters...")
    agent.load_model(filepath=full_model_path)

  optimizer = torch.optim.Adam(agent.parameters(), lr=LEARNING_RATE)
  criterion = torch.nn.SmoothL1Loss() # Huber Loss

  return agent, optimizer, criterion

def compare_to_naive(agent_name: str, num_episodes: int = 10000):
  _agent, _, _ = supply_model(load_if_exists=True, agent_name=agent_name)
  _environment = TicTacToeGame(DEVICE, None, OPPONENT_LEVEL.NAIVE, start_as_X=False)
  test_agent(_agent, _environment, num_episodes)

def compare_to_optimal(agent_name: str, num_episodes: int = 100):
  _agent, _, _ = supply_model(load_if_exists=True, agent_name=agent_name)
  _environment = TicTacToeGame(DEVICE, None, OPPONENT_LEVEL.OPTIMAL, start_as_X=False)
  test_agent(_agent, _environment, num_episodes)

def compare_to_model(agent_name: str, model_name: str, num_episodes: int = 10000):
  _agent, _, _ = supply_model(load_if_exists=True, agent_name=agent_name)
  _enemy, _, _ = supply_model(load_if_exists=True, agent_name=model_name)
  _environment = TicTacToeGame(DEVICE, _enemy, OPPONENT_LEVEL.AGENT, start_as_X=False)
  test_agent(_agent, _environment, num_episodes)


cuda


In [ ]:
agent, _, _ = supply_model()
agent.save_model(MODEL_PATH, BASELINE)

Model saved to 'drive/MyDrive/Projects/artificial-intelligence/trained_models/ReinforcementLearning/TicTacToeV2/TicTacToe-v2-BASELINE.pt'.


'drive/MyDrive/Projects/artificial-intelligence/trained_models/ReinforcementLearning/TicTacToeV2/TicTacToe-v2-BASELINE.pt'

In [ ]:
compare_to_naive(BASELINE)

Loading Model Parameters...
Model loaded from 'drive/MyDrive/Projects/artificial-intelligence/trained_models/ReinforcementLearning/TicTacToeV2/TicTacToe-v2-BASELINE.pt'.


100%|██████████| 10000/10000 [01:08<00:00, 146.06it/s]


Win rate:  54.13%
Draw rate: 8.19%
Loss rate: 37.68%


In [5]:
NUM_EPISODES = 10000
TRAIN_START  = 2500

In [ ]:
agent, optimizer, criterion = supply_model(load_if_exists=True, agent_name=BASELINE)
environment = TicTacToeGame(DEVICE, None, OPPONENT_LEVEL.NAIVE)
reward_history = train_agent(
    agent,
    environment,
    NUM_EPISODES,
    optimizer,
    criterion,
    DEVICE,
    MODEL_PATH,
    model_name = NAIVE,
    save_every = 2000,
    epsilon_min_value = 0.30,
    epsilon_max_value = 0.75,
)

Streaming output truncated to the last 5000 lines.
|O|O|X|
‾‾‾‾‾‾‾
episode: 6983/20000, steps: 3, reward_total: 1, e: 0.51, time: 11:21:23
_______
|O| |X|
|X|O| |
| |X|O|
‾‾‾‾‾‾‾
episode: 6984/20000, steps: 5, reward_total: -1, e: 0.51, time: 11:21:28
_______
|X|O|O|
|O|X|X|
|X|X|O|
‾‾‾‾‾‾‾
episode: 6985/20000, steps: 4, reward_total: 1, e: 0.5, time: 11:21:32
_______
|X|X|O|
| |X|X|
|O|O|O|
‾‾‾‾‾‾‾
episode: 6986/20000, steps: 5, reward_total: 1, e: 0.5, time: 11:21:37
_______
|O|O|X|
|O|X|X|
|X|O|X|
‾‾‾‾‾‾‾
episode: 6987/20000, steps: 3, reward_total: -2, e: 0.5, time: 11:21:40
_______
|X|O|X|
|X|O| |
|X| |O|
‾‾‾‾‾‾‾
episode: 6988/20000, steps: 4, reward_total: 1, e: 0.5, time: 11:21:45
_______
|X|O|O|
|X|O| |
|X| |X|
‾‾‾‾‾‾‾
episode: 6989/20000, steps: 2, reward_total: -2, e: 0.5, time: 11:21:47
_______
|O| |X|
| |O|X|
| | |X|
‾‾‾‾‾‾‾
episode: 6990/20000, steps: 5, reward_total: -1, e: 0.49, time: 11:21:52
_______
|O|X|O|
|O|X|X|
|X|O|X|
‾‾‾‾‾‾‾
episode: 6991/20000, steps: 3, reward_

In [ ]:
compare_to_naive(f"{NAIVE}-2K")

Loading Model Parameters...
Model loaded from 'drive/MyDrive/Projects/artificial-intelligence/trained_models/ReinforcementLearning/TicTacToeV2/TicTacToe-v2-NAIVE-2K.pt'.


100%|██████████| 10000/10000 [01:08<00:00, 146.33it/s]


Win rate:  68.4%
Draw rate: 5.4%
Loss rate: 26.2%


In [ ]:
compare_to_naive(f"{NAIVE}-4K")

Loading Model Parameters...
Model loaded from 'drive/MyDrive/Projects/artificial-intelligence/trained_models/ReinforcementLearning/TicTacToeV2/TicTacToe-v2-NAIVE-4K.pt'.


100%|██████████| 10000/10000 [01:06<00:00, 151.16it/s]


Win rate:  65.01%
Draw rate: 10.64%
Loss rate: 24.35%


In [ ]:
compare_to_naive(f"{NAIVE}-6K")

Loading Model Parameters...
Model loaded from 'drive/MyDrive/Projects/artificial-intelligence/trained_models/ReinforcementLearning/TicTacToeV2/TicTacToe-v2-NAIVE-6K.pt'.


100%|██████████| 10000/10000 [00:59<00:00, 167.73it/s]


Win rate:  69.66%
Draw rate: 7.89%
Loss rate: 22.45%


In [ ]:
compare_to_naive(f"{NAIVE}-8K")

Loading Model Parameters...
Model loaded from 'drive/MyDrive/Projects/artificial-intelligence/trained_models/ReinforcementLearning/TicTacToeV2/TicTacToe-v2-NAIVE-8K.pt'.


100%|██████████| 10000/10000 [00:59<00:00, 167.85it/s]


Win rate:  68.87%
Draw rate: 9.61%
Loss rate: 21.52%


In [ ]:
compare_to_naive(f"{NAIVE}-10K")

Loading Model Parameters...
Model loaded from 'drive/MyDrive/Projects/artificial-intelligence/trained_models/ReinforcementLearning/TicTacToeV2/TicTacToe-v2-NAIVE-10K.pt'.


100%|██████████| 10000/10000 [01:00<00:00, 165.69it/s]


Win rate:  68.1%
Draw rate: 6.04%
Loss rate: 25.86%


In [ ]:
# The NAIVE-10+ model has trained for
# 2000 extra episodes to account for
# episodes not trained upon before TRAIN start
compare_to_naive(f"{NAIVE}-10K+")

Loading Model Parameters...
Model loaded from 'drive/MyDrive/Projects/artificial-intelligence/trained_models/ReinforcementLearning/TicTacToeV2/TicTacToe-v2-NAIVE-10K+.pt'.


100%|██████████| 10000/10000 [01:04<00:00, 156.18it/s]


Win rate:  70.68%
Draw rate: 4.13%
Loss rate: 25.19%
